## Get and load data

In [1]:
import os
import gzip
import shutil
from Bio import SeqIO
import pandas as pd
import numpy as np
from tqdm import tqdm

# --- CẤU HÌNH ---
DATA_DIR = "/workspace/data/External"
os.makedirs(DATA_DIR, exist_ok=True)

# Link tải Swiss-Prot (File FASTA chứa trình tự)
SPROT_URL = "https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.fasta.gz"
SPROT_GZ = f"{DATA_DIR}/uniprot_sprot.fasta.gz"
SPROT_FASTA = f"{DATA_DIR}/uniprot_sprot.fasta"

# 1. Tải file (Khoảng 90MB nén -> 300MB giải nén)
if not os.path.exists(SPROT_FASTA):
    print("Đang tải Swiss-Prot từ UniProt FTP...")
    os.system(f"wget -q {SPROT_URL} -O {SPROT_GZ}")
    
    print("Đang giải nén...")
    with gzip.open(SPROT_GZ, 'rb') as f_in:
        with open(SPROT_FASTA, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    print("Đã tải xong file Fasta!")
else:
    print("File Swiss-Prot đã có sẵn.")

# --- LỌC TRÙNG (QUAN TRỌNG) ---
# Ta không muốn chạy embedding lại cho 142k protein đã có trong Kaggle Train.
# Hãy lọc chúng ra.

# Load Kaggle Train IDs
print("Đang load danh sách ID của Kaggle Train...")
train_ids = np.load("/workspace/data/Embeddings/train_650M_ids.npy", allow_pickle=True)
kaggle_ids_set = set(train_ids)

# Tạo file Fasta mới chỉ chứa protein "LẠ"
NEW_FASTA = f"{DATA_DIR}/new_sprot_sequences.fasta"
new_ids = []

print(f"Đang lọc protein mới và ghi vào {NEW_FASTA}...")
count = 0
with open(NEW_FASTA, "w") as f_out:
    for record in tqdm(SeqIO.parse(SPROT_FASTA, "fasta")):
        # Header UniProt: >sp|Q8K9I1|...
        pid = str(record.id).split("|")[1] if "|" in str(record.id) else str(record.id)
        
        # Nếu protein này CHƯA có trong Kaggle -> Giữ lại
        if pid not in kaggle_ids_set:
            # Ghi vào file
            SeqIO.write(record, f_out, "fasta")
            new_ids.append(pid)
            count += 1

print(f"Đã lọc xong! Tìm thấy {count} protein mới (Ngoài 142k cái cũ).")
print(f"File này sẽ được dùng để chạy Embedding.")

Đang tải Swiss-Prot từ UniProt FTP...
Đang giải nén...
Đã tải xong file Fasta!
Đang load danh sách ID của Kaggle Train...
Đang lọc protein mới và ghi vào /workspace/data/External/new_sprot_sequences.fasta...


573661it [00:02, 245402.44it/s]

Đã lọc xong! Tìm thấy 491257 protein mới (Ngoài 142k cái cũ).
File này sẽ được dùng để chạy Embedding.


## Chunking

In [2]:
import os
import gc
import torch
import numpy as np
from Bio import SeqIO
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel

# --- CẤU HÌNH ---
FASTA_PATH = "/workspace/data/External/new_sprot_sequences.fasta" # File mới tạo
SAVE_DIR = "/workspace/data/External/embeddings_chunks"           # Lưu riêng ra folder khác
MODEL_NAME = "facebook/esm2_t33_650M_UR50D"

CHUNK_SIZE = 5000  
BATCH_SIZE = 8    

os.makedirs(SAVE_DIR, exist_ok=True)
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Đang load model {MODEL_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME).to(device)
model.eval()

# Hàm xử lý (Y hệt cũ)
def process_and_save(seqs, ids, part_idx):
    embeddings = []
    for i in range(0, len(seqs), BATCH_SIZE):
        batch_seqs = seqs[i : i + BATCH_SIZE]
        inputs = tokenizer(batch_seqs, return_tensors="pt", padding=True, truncation=True, max_length=1024)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        with torch.no_grad():
            outputs = model(**inputs)
            last_hidden_state = outputs.last_hidden_state
            mask = inputs['attention_mask'].unsqueeze(-1).expand(last_hidden_state.size()).float()
            sum_embeddings = torch.sum(last_hidden_state * mask, 1)
            sum_mask = torch.clamp(mask.sum(1), min=1e-9)
            mean_embeddings = sum_embeddings / sum_mask
            
        embeddings.append(mean_embeddings.cpu().numpy())
    
    final_emb = np.vstack(embeddings)
    np.save(f"{SAVE_DIR}/sprot_part_{part_idx}.npy", final_emb)
    np.save(f"{SAVE_DIR}/sprot_ids_{part_idx}.npy", ids)

# --- VÒNG LẶP CHÍNH ---
sequences = []
ids = []
part_counter = 0

print(f"Bắt đầu Embedding Swiss-Prot (Đây là bước tốn thời gian nhất)...")
# Đếm số dòng (ước lượng)
# Bạn vừa in ra biến 'count' ở cell trên, thay số đó vào total nếu muốn thanh progress chuẩn
total_seqs = 427000 # Ước lượng
pbar = tqdm(total=total_seqs)

for record in SeqIO.parse(FASTA_PATH, "fasta"):
    # Resume Logic
    save_path_check = f"{SAVE_DIR}/sprot_part_{part_counter}.npy"
    if os.path.exists(save_path_check):
        sequences.append(1)
        if len(sequences) >= CHUNK_SIZE:
            sequences = [] 
            ids = []
            part_counter += 1
            pbar.update(CHUNK_SIZE)
        continue

    pid = str(record.id).split("|")[1] if "|" in str(record.id) else str(record.id)
    ids.append(pid)
    # Cắt 1022
    sequences.append(str(record.seq)[:1022])
    pbar.update(1)
    
    if len(sequences) >= CHUNK_SIZE:
        process_and_save(sequences, ids, part_counter)
        part_counter += 1
        sequences = []
        ids = []
        gc.collect()

if len(sequences) > 0:
    process_and_save(sequences, ids, part_counter)

print("\SIÊU DỮ LIỆU ĐÃ SẴN SÀNG!")

<>:86: SyntaxWarning: invalid escape sequence '\S'
<>:86: SyntaxWarning: invalid escape sequence '\S'
/tmp/ipykernel_742/3438287253.py:86: SyntaxWarning: invalid escape sequence '\S'
  print("\SIÊU DỮ LIỆU ĐÃ SẴN SÀNG!")


Đang load model facebook/esm2_t33_650M_UR50D...


Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t33_650M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Bắt đầu Embedding Swiss-Prot (Đây là bước tốn thời gian nhất)...


491257it [2:11:48, 58.31it/s]                              

\SIÊU DỮ LIỆU ĐÃ SẴN SÀNG!


In [13]:
# --- BƯỚC 3: CHẠY KNN CUỐN CHIẾU ---
test_files = sorted(glob.glob(f"{TEST_CHUNKS_DIR}/test_part_*.npy"), 
                    key=lambda x: int(x.split('_')[-1].replace('.npy','')))

output_lines = []

# Duyệt qua từng cục Test
print("Bắt đầu quét...")
for test_f in tqdm(test_files, desc="Test Chunks"):
    # 1. Load 1 cục Test lên GPU
    X_test_np = np.load(test_f)
    ids_test = np.load(test_f.replace("test_part_", "test_ids_"), allow_pickle=True)
    
    X_test = torch.from_numpy(X_test_np).to(device)
    norm_test = X_test.norm(p=2, dim=1, keepdim=True)
    X_test = X_test.div(norm_test)
    
    # Biến lưu trữ Top K ứng viên tạm thời cho batch test này
    # Cấu trúc list of list: candidates[i] = [(score, pid), (score, pid)...]
    candidates = [[] for _ in range(len(ids_test))]
    
    # 2. Quét qua từng cục Train (Streaming)
    for train_chunk in train_chunks:
        # Load cục Train này lên RAM -> GPU
        X_train_np = np.load(train_chunk["emb"])
        ids_train = np.load(train_chunk["ids"], allow_pickle=True)
        
        X_train = torch.from_numpy(X_train_np).to(device)
        norm_train = X_train.norm(p=2, dim=1, keepdim=True)
        X_train = X_train.div(norm_train)
        
        # Tính Sim
        sim_matrix = torch.mm(X_test, X_train.t())
        
        # Lấy Top K của riêng cục này
        # (Lấy nhiều hơn chút đề phòng trùng lặp, ví dụ K=20)
        curr_k = min(TOP_K, X_train.size(0))
        topk_vals, topk_idxs = torch.topk(sim_matrix, k=curr_k, dim=1)
        
        topk_vals = topk_vals.cpu().numpy()
        topk_idxs = topk_idxs.cpu().numpy()
        
        # Cập nhật vào danh sách ứng viên
        for i in range(len(ids_test)):
            for k in range(curr_k):
                score = topk_vals[i, k]
                train_idx = topk_idxs[i, k]
                train_pid = ids_train[train_idx]
                
                candidates[i].append((score, train_pid))
        
        # Dọn dẹp GPU ngay sau khi dùng xong cục train này
        del X_train, sim_matrix, topk_vals, topk_idxs, X_train_np, ids_train
        torch.cuda.empty_cache()
        
    # 3. Tổng hợp kết quả cho cục Test này
    # Sau khi quét hết các file train, ta sort lại và lấy Top K xịn nhất
    for i, test_pid in enumerate(ids_test):
        # Sort giảm dần theo score
        my_candidates = sorted(candidates[i], key=lambda x: x[0], reverse=True)
        
        # Lấy Top K ông hàng xóm tốt nhất toàn cục
        best_neighbors = my_candidates[:TOP_K]
        
        # Voting Logic
        term_scores = {}
        for score, neighbor_pid in best_neighbors:
            if neighbor_pid in labels_map:
                for term in labels_map[neighbor_pid]:
                    if term not in term_scores: term_scores[term] = 0.0
                    term_scores[term] += score
        
        # Ghi kết quả
        sorted_terms = sorted(term_scores.items(), key=lambda x: x[1], reverse=True)[:60]
        for term, total_score in sorted_terms:
            final_score = total_score / TOP_K
            if final_score > 0.01:
                output_lines.append(f"{test_pid}\t{term}\t{final_score:.3f}")

    # Xóa cục Test
    del X_test, X_test_np, ids_test, candidates
    gc.collect()

Bắt đầu quét...



Test Chunks: 100%|██████████| 45/45 [03:40<00:00,  4.90s/it]


In [17]:
import torch
import numpy as np
import pandas as pd
import gc
from tqdm import tqdm
import glob
import os
import networkx
import obonet

# ================= CẤU HÌNH ĐƯỜNG DẪN =================
# 1. Dữ liệu Train (Kaggle + External)
KAGGLE_TRAIN_EMB = "/workspace/data/Embeddings/train_650M.npy"
KAGGLE_TRAIN_IDS = "/workspace/data/Embeddings/train_650M_ids.npy"
KAGGLE_TERMS     = "/workspace/data/Train/train_terms.tsv"

EXT_CHUNKS_DIR   = "/workspace/data/External/embeddings_chunks"
SPROT_TERMS_FILE = "/workspace/data/External/uniprot_sprot_terms.tsv"

# 2. Dữ liệu Test
TEST_CHUNKS_DIR  = "/workspace/data/Embeddings/embeddings_chunks"

# 3. File Cây phả hệ (OBO)
# Kiểm tra xem file nằm ở đâu
if os.path.exists("/workspace/data/Train/go-basic.obo"):
    OBO_PATH = "/workspace/data/Train/go-basic.obo"
else:
    OBO_PATH = "/workspace/data/go-basic.obo"

# 4. File đầu ra cuối cùng
FINAL_OUTPUT = "submission_level10_complete.tsv"

# 5. Tham số
TOP_K = 15 
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"⚙️ Thiết bị: {device}")
print(f"📂 Output sẽ lưu tại: {FINAL_OUTPUT}")

# ================= PHẦN 1: LOAD NHÃN (LABELS) =================
print("\n[1/4] ⏳ Đang load bản đồ nhãn (Label Map)...")
labels_map = {}

# A. Load Kaggle Labels
print("   -> Loading Kaggle Labels...")
df_k = pd.read_csv(KAGGLE_TERMS, sep="\t", usecols=["EntryID", "term"])
# Update vào dict
for pid, group in df_k.groupby("EntryID"):
    labels_map[pid] = set(group["term"].values)
del df_k
gc.collect()

# B. Load Swiss-Prot Labels (Streaming)
print("   -> Parsing Swiss-Prot Labels...")
if os.path.exists(SPROT_TERMS_FILE):
    chunksize = 200000
    for chunk in pd.read_csv(SPROT_TERMS_FILE, sep="\t", chunksize=chunksize):
        for _, row in chunk.iterrows():
            pid = row.iloc[0]
            go_string = str(row.iloc[1])
            if go_string == "nan": continue
            terms = set([t.strip() for t in go_string.split(";") if t.strip().startswith("GO:")])
            if len(terms) > 0:
                labels_map[pid] = terms
else:
    print("⚠️ Cảnh báo: Không thấy file nhãn Swiss-Prot!")

print(f"✅ Đã load nhãn cho {len(labels_map)} protein.")

# ================= PHẦN 2: CHUẨN BỊ DANH SÁCH FILE =================
print("\n[2/4] ⏳ Chuẩn bị danh sách file Train/Test...")
train_chunks = []

# Thêm Kaggle Train gốc
if os.path.exists(KAGGLE_TRAIN_EMB):
    train_chunks.append({"emb": KAGGLE_TRAIN_EMB, "ids": KAGGLE_TRAIN_IDS})

# Thêm External Chunks
sprot_files = sorted(glob.glob(f"{EXT_CHUNKS_DIR}/sprot_part_*.npy"))
for f in sprot_files:
    train_chunks.append({"emb": f, "ids": f.replace("sprot_part_", "sprot_ids_")})

print(f"   -> Tổng cộng: {len(train_chunks)} mảnh dữ liệu Train.")

# Lấy danh sách Test
test_files = sorted(glob.glob(f"{TEST_CHUNKS_DIR}/test_part_*.npy"), 
                    key=lambda x: int(x.split('_')[-1].replace('.npy','')))
if len(test_files) == 0:
    raise FileNotFoundError(f"Không thấy file Test nào trong {TEST_CHUNKS_DIR}")

# ================= PHẦN 3: CHẠY KNN STREAMING =================
print("\n[3/4] 🚀 Bắt đầu quét KNN (Quá trình này mất khoảng 30-45 phút)...")

# Biến lưu kết quả thô: {PID: {Term: Score}}
raw_predictions = {}

for test_f in tqdm(test_files, desc="Processing Test Chunks"):
    # 1. Load 1 cục Test
    X_test_np = np.load(test_f)
    ids_test = np.load(test_f.replace("test_part_", "test_ids_"), allow_pickle=True)
    
    X_test = torch.from_numpy(X_test_np).to(device)
    norm_test = X_test.norm(p=2, dim=1, keepdim=True)
    X_test = X_test.div(norm_test)
    
    # List lưu ứng viên: candidates[i] = [(score, pid), ...]
    candidates = [[] for _ in range(len(ids_test))]
    
    # 2. Quét qua TOÀN BỘ Train Chunks
    for train_chunk in train_chunks:
        try:
            X_train_np = np.load(train_chunk["emb"])
            ids_train = np.load(train_chunk["ids"], allow_pickle=True)
        except:
            continue 
            
        X_train = torch.from_numpy(X_train_np).to(device)
        norm_train = X_train.norm(p=2, dim=1, keepdim=True)
        X_train = X_train.div(norm_train)
        
        # Tính Sim
        sim_matrix = torch.mm(X_test, X_train.t())
        
        # Lấy Top K cục bộ
        curr_k = min(TOP_K, X_train.size(0))
        topk_vals, topk_idxs = torch.topk(sim_matrix, k=curr_k, dim=1)
        
        topk_vals = topk_vals.cpu().numpy()
        topk_idxs = topk_idxs.cpu().numpy()
        
        # Lưu lại
        for i in range(len(ids_test)):
            for k in range(curr_k):
                candidates[i].append((topk_vals[i, k], ids_train[topk_idxs[i, k]]))
        
        del X_train, sim_matrix, topk_vals, topk_idxs, X_train_np, ids_train
        torch.cuda.empty_cache()
    
    # 3. Tổng hợp & Voting cho cục Test này
    for i, test_pid in enumerate(ids_test):
        # Sort lấy Top K toàn cục
        best_neighbors = sorted(candidates[i], key=lambda x: x[0], reverse=True)[:TOP_K]
        
        term_scores = {}
        for score, neighbor_pid in best_neighbors:
            if neighbor_pid in labels_map:
                for term in labels_map[neighbor_pid]:
                    if term not in term_scores: term_scores[term] = 0.0
                    term_scores[term] += score
        
        # Normalize score (chia cho K)
        final_term_scores = {t: s/TOP_K for t, s in term_scores.items()}
        raw_predictions[test_pid] = final_term_scores

    del X_test, X_test_np, ids_test, candidates
    gc.collect()

print(f"✅ KNN Xong! Đã dự đoán cho {len(raw_predictions)} protein.")

# ================= PHẦN 4: PROPAGATION & GHI FILE =================
print("\n[4/4] 🚀 Đang Lan truyền điểm (Propagation) & Ghi File...")

# Load OBO
if not os.path.exists(OBO_PATH):
    print("⚠️ Đang tải OBO file...")
    os.system("wget -q http://purl.obolibrary.org/obo/go/go-basic.obo -O go-basic.obo")
    OBO_PATH = "go-basic.obo"

graph = obonet.read_obo(OBO_PATH)
ancestors_map = {}
for node in tqdm(graph.nodes(), desc="Mapping Ancestors"):
    try:
        ancestors_map[node] = networkx.descendants(graph, node)
    except:
        pass

output_lines = []

# Duyệt qua từng protein đã dự đoán
for pid, term_scores in tqdm(raw_predictions.items(), desc="Propagating"):
    new_scores = term_scores.copy()
    
    # Logic lan truyền
    for term, score in term_scores.items():
        if term in ancestors_map:
            parents = ancestors_map[term]
            for parent in parents:
                current_p_score = new_scores.get(parent, 0.0)
                if score > current_p_score:
                    new_scores[parent] = score
    
    # Lọc Top 75 để file nhẹ
    sorted_terms = sorted(new_scores.items(), key=lambda x: x[1], reverse=True)
    count = 0
    for term, score in sorted_terms:
        if score > 0.01:
            output_lines.append(f"{pid}\t{term}\t{score:.3f}")
            count += 1
        if count >= 75: break

# Ghi file
print(f"💾 Đang lưu xuống {FINAL_OUTPUT}...")
with open(FINAL_OUTPUT, "w") as f:
    f.write("\n".join(output_lines))

print("\n" + "="*40)
print(f"🎉 HOÀN TẤT! File '{FINAL_OUTPUT}' đã sẵn sàng.")
print("👉 Hãy nộp file này. Đây là kết quả tối ưu nhất!")

⚙️ Thiết bị: cuda
📂 Output sẽ lưu tại: submission_level10_complete.tsv

[1/4] ⏳ Đang load bản đồ nhãn (Label Map)...
   -> Loading Kaggle Labels...
   -> Parsing Swiss-Prot Labels...
✅ Đã load nhãn cho 553381 protein.

[2/4] ⏳ Chuẩn bị danh sách file Train/Test...
   -> Tổng cộng: 100 mảnh dữ liệu Train.

[3/4] 🚀 Bắt đầu quét KNN (Quá trình này mất khoảng 30-45 phút)...



Processing Test Chunks: 100%|██████████| 45/45 [03:31<00:00,  4.70s/it]


✅ KNN Xong! Đã dự đoán cho 224309 protein.

[4/4] 🚀 Đang Lan truyền điểm (Propagation) & Ghi File...



Mapping Ancestors: 100%|██████████| 40122/40122 [00:00<00:00, 53198.80it/s]

Propagating: 100%|██████████| 224309/224309 [00:20<00:00, 11024.09it/s]


💾 Đang lưu xuống submission_level10_complete.tsv...

🎉 HOÀN TẤT! File 'submission_level10_complete.tsv' đã sẵn sàng.
👉 Hãy nộp file này. Đây là kết quả tối ưu nhất!


In [18]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

# --- CẤU HÌNH ---
# 1. File dự đoán tốt nhất hiện có (File Level 10 đã Propagation - 0.249)
INPUT_FILE = "submission_level10_complete.tsv"

# 2. File trọng số chuẩn của BTC
IA_FILE = "/workspace/data/IA.tsv"

# 3. File đầu ra
OUTPUT_FILE = "submission_level11_IA_official.tsv"

# --- BƯỚC 1: LOAD FILE IA.TSV ---
print(f"📖 Đang đọc bảng trọng số chuẩn từ {IA_FILE}...")

# File IA.tsv thường không có header, cột 1 là Term, cột 2 là IA Score
# Nếu file nằm trong thư mục Train thì sửa đường dẫn lại nhé
if not os.path.exists(IA_FILE):
    # Thử tìm trong thư mục Train nếu không thấy ở ngoài
    IA_FILE = "/workspace/data/Train/IA.tsv"

try:
    # Đọc file IA
    df_ia = pd.read_csv(IA_FILE, sep="\t", names=["term", "ia_score"])
    
    # Chuyển thành Dict cho nhanh: {GO:0001 : 5.23}
    ia_weights = dict(zip(df_ia["term"], df_ia["ia_score"]))
    print(f"✅ Đã load trọng số cho {len(ia_weights)} nhãn.")
    
except FileNotFoundError:
    print("❌ LỖI: Không tìm thấy file IA.tsv! Bạn check lại xem nó nằm ở folder data gốc hay data/Train?")
    raise

# --- BƯỚC 2: BƠM ĐIỂM DỰA TRÊN TRỌNG SỐ CHUẨN ---
print(f"🚀 Đang boost điểm cho {INPUT_FILE}...")
output_lines = []

# Hệ số boost (Thử mức nhẹ nhàng trước)
BOOST_FACTOR = 0.02 
THRESHOLD = 0.015 

with open(INPUT_FILE) as f:
    for line in tqdm(f):
        parts = line.strip().split('\t')
        if len(parts) < 3: continue
        pid, term, score = parts[0], parts[1], float(parts[2])
        
        # Lấy trọng số từ file IA
        # Nếu nhãn không có trong file IA (nhãn lạ), ta cho mặc định là 0 (không boost)
        weight = ia_weights.get(term, 0.0)
        
        # Công thức Boost: Score * (1 + hệ_số * độ_hiếm)
        # Nhãn càng hiếm (IA cao), điểm càng được nhân lên nhiều
        new_score = score * (1 + BOOST_FACTOR * weight)
        new_score = min(new_score, 1.0)
        
        # Lọc ngưỡng
        if new_score > THRESHOLD:
            output_lines.append(f"{pid}\t{term}\t{new_score:.3f}")

# --- BƯỚC 3: LƯU FILE ---
print(f"💾 Đang lưu {OUTPUT_FILE}...")
with open(OUTPUT_FILE, "w") as f:
    f.write("\n".join(output_lines))

print("🎉 XONG! Đây mới là cách dùng trọng số 'chuẩn bài' nhất.")

📖 Đang đọc bảng trọng số chuẩn từ /workspace/data/IA.tsv...
✅ Đã load trọng số cho 40122 nhãn.
🚀 Đang boost điểm cho submission_level10_complete.tsv...



0it [00:00, ?it/s]
112935it [00:00, 1129332.23it/s]
230673it [00:00, 1157587.97it/s]
346733it [00:00, 1158939.85it/s]
462627it [00:00, 1153247.02it/s]
583442it [00:00, 1172976.30it/s]
700749it [00:00, 1151758.40it/s]
828186it [00:00, 1191292.92it/s]
967163it [00:00, 1253884.94it/s]
1099450it [00:00, 1275307.24it/s]
1236092it [00:01, 1303306.42it/s]
1367548it [00:01, 1306737.42it/s]
1498446it [00:01, 1307414.75it/s]
1629229it [00:01, 1307453.99it/s]
1767649it [00:01, 1330606.01it/s]
1903456it [00:01, 1338874.65it/s]
2037360it [00:01, 1329407.69it/s]
2170328it [00:01, 1325677.06it/s]
2303024it [00:01, 1326055.06it/s]
2436644it [00:01, 1329086.30it/s]
2570810it [00:02, 1332847.31it/s]
2704104it [00:02, 1328665.78it/s]
2836979it [00:02, 1327720.12it/s]
2969757it [00:02, 1327566.07it/s]
3104234it [00:02, 1332710.52it/s]
3243809it [00:02, 1351590.08it/s]
3383844it [00:02, 1366201.89it/s]
3520470it [00:02, 1353818.31it/s]
3658025it [00:02, 1360288.66it/s]
3794079it [00:02, 1349690.41it/s]
39

💾 Đang lưu submission_level11_IA_official.tsv...
🎉 XONG! Đây mới là cách dùng trọng số 'chuẩn bài' nhất.
